### Loading the functions

In [ ]:
# Hands-on 1
from unstructured_rag import get_list_from_pkl, get_pieces_of_interest, ask_question

# Hands-on 2
from structured_rag import parse_island_name, create_request_for_text2sql, generate_sql_query, fetch_data_from_database, clean_generated_sql

# Hands-on 3
from orchestrator import controller, acceptance

import sqlite3

# Welcome to our Travel Agency.

<img src="images/9_travel_agent.png" alt="Image description" width="1000">  

Now that we have all the functions loaded we put them together as a travel agent

In [ ]:
def travel_agent(question:str):

    ### Hands-on 3 | Orchestrator ###
    print("🔍 Starting Orchestration...")
    identified_consideration = controller(question, acceptance, "gpt-4")
    consideration = "complete"
    if identified_consideration.lower() != consideration:
        return(identified_consideration)
    else:
        print("✅ Consideration matched, proceeding with tasks...")


    ### Hands-on 1 | Information Retrieval: Semantic search for best island ###
    print("🌴 1/Start - Retrieving island description...")

    best_fitting_island = get_pieces_of_interest(question=question,
                                                embeddings=get_list_from_pkl("./data/embeddings.pkl"),
                                                travel_articles=get_list_from_pkl("./data/travel_articles.pkl"),
                                                k=1)
    print("🌴 1/End - Found best fitting island!")



    ### Hands-on 2 | Information Retrieval: SQL Query for travel details ###
    print("🏝️ 2/Start - Retrieve travel infromation...")
    island_name = parse_island_name(best_fitting_island[0])
    print(f"📝 2 - Creating SQL query to get travel infromation to {island_name}")
    text2sql_query = create_request_for_text2sql(island_name)
    text2sql_query = text2sql_query + "Please consider the following statement only if it contains relevant information: " + question 
    print("🧠 Generating SQL query...")
    sql_query = generate_sql_query(text2sql_query)
    print("📊 Fetching travel information...")
    conn = sqlite3.connect("data/travel_information.sqlite")
    travel_information = fetch_data_from_database(sql_query, conn) 

    print("📊 2/End Fetched travel information...")



    ### Travel Agent - Augmented Answer Generation: LLM to generate Answer ###
    print("💡 Generating augmented answer...")

    travel_agent_character = """You are a seasoned travel agent with the primary goal to help users looking to plan a vacation in Greece.
                                You only include information provided in the prompt nothing else.
                                """

    augmented_question_with_retrieved_information = f"""Use the travel information below to give an offer for a holiday trip: \n
            1. Customer request, this will be what you have to answer: \n
                {question} \n \n
            2. Infromation about the island the customer wants to go. Highlight the parts that are most fitting for the customer request: \n
                {best_fitting_island[0]} \n \n
            3. Travel information how to get to the island from Athens. If the client indicates a preferred way of travel highlight this in the response: \n
                {str(travel_information.to_json(orient='records', lines=False))} \n \n
        
            Output:
            
            The generated answer should be in the following paragraphs, clearly indicate the following sections:
            Why you will love this: One sentence description why the island fits the customer request
            Highlights: Relevant details about the island the customer might be interested in
            Travel Options: Travel option, highlighting the clients needs.
            """
    

    prompt = [
    {
        "role": "system",
        "content": travel_agent_character
    },
    {
        "role": "user",
        "content": augmented_question_with_retrieved_information
    },
    ]


    response = ask_question(prompt, model="gpt-35-turbo")
    print("🎉 Ready for vacation? Consider: ")
    return(response.choices[0].message.content)

In [ ]:
question = # "I want to do a trip to the islands of greece for two weeks"
response = travel_agent(question)
print(response)

Lets have fun:


- Change the travel_agent_character to represent different `personas`, such as an adventure travel agent, a luxury travel agent, or a budget travel agent. Observe how the character influences the generated answers.
- How would you optimize the `augmented_question_with_retrieved_information`
- Do you think the controller keeps working with `gpt-35-turbo`?
- Given what you've seen in todays workshop `structured RAG`, `unstructured RAG`, a `controller` - how would you bring this application to the next level?